In [3]:
# importing the usual suspects
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from multiprocessing import Pool, cpu_count
from math import floor

# random number generation
import random
import fractions
# i just picked an arbitrary seed here
# if it matters it's easy to change
random.seed("snoplus")

plt.rcParams['figure.figsize'] = (12, 8)

In [4]:
# copying this from the main version
# now we read in the stopping power data from ASTAR
# using dry air in this case since we are re-creating the rutherford experiment
stoppingpowers = pd.read_csv(Path("stoppingpowers/dryair.csv"))
# rename cols to make them easier to reference
stoppingpowers.columns = ["KE", "electron", "nuclear", "total"]
# reindex the dataframe by kinetic energy to make lookups easier
#stoppingpowers.set_index("KE", inplace=True)
print(stoppingpowers)

            KE  electron    nuclear  total
0       0.0010      87.5  134.00000  222.0
1       0.0015     109.0  126.00000  234.0
2       0.0020     127.0  118.00000  244.0
3       0.0025     143.0  111.00000  254.0
4       0.0030     157.0  105.00000  262.0
..         ...       ...        ...    ...
116   800.0000      15.8    0.00453   15.8
117   850.0000      15.2    0.00427   15.2
118   900.0000      14.7    0.00404   14.7
119   950.0000      14.2    0.00382   14.2
120  1000.0000      13.8    0.00364   13.8

[121 rows x 4 columns]


In [7]:
def scattering_probability(ke, dx):
    sample_dim = 1 # cm

    N_A = 6.02214E23 # mol^-1
    mol_wt = 200
    rho = 19.3 # g/cm^3, see above
    #n = ((N_A/mol_wt) * rho)*(sample_dim**2)*dx
    n = N_A/(mol_wt) * dx * rho
    # effective area based on cross-section
    Z = 79 
    D = ((197.3/137) * (2*Z)/ke)*1E-15
    # the 140.646 comes from integrating the differential cx expression in
    # 02_rutherford
    # here we only account for scattering > 10 degrees (around 0.1745 rad)
    sigma = 130.646*4*np.pi*np.power(D, 2)
    #print(sigma)
    #print(sigma)
    #sigma = 100 * 1E-24
    #sigma = 100 * 1E-15
    # total particle effective area
    eff_a = sigma*n

    # the total area
    total_a = sample_dim**2
    return eff_a/total_a

# note to self - alpha energy is 4.7 mev
#s_xrange = np.linspace(5, 10, 10)
#s_probs = scattering_probability(s_xrange, 1E-4)
#_probsfrac = [str(fractions.Fraction(i)) for i in s_probs]

#print(s_probs)
print(scattering_probability(6, 0.0001))


0.0005811365099999999


In [77]:
# now to do the linear interpolation for the ASTAR dataset
def stp_interp(e, stp=stoppingpowers):
    # NOTE this assumes that the stopping powers are sorted
    # we get them this way from ASTAR so it's not an issue, but we can fix that if need be
    for k in stoppingpowers.index:
        if e >= stoppingpowers["KE"].max():
            return (stoppingpowers["total"].iloc(-1))/(stoppingpowers["KE"].iloc(-1))*e
        elif e <= stoppingpowers["KE"][k+1] and e >= stoppingpowers["KE"][k]:
            ke_low = stoppingpowers["KE"][k]
            sp_low = stoppingpowers["total"][k]
            ke_high = stoppingpowers["KE"][k+1]
            sp_high = stoppingpowers["total"][k+1]
            break

    return ((sp_high - sp_low)/(ke_high - ke_low)) * e + sp_low

In [34]:
def scatter_sim(e_0, epsilon=0.1, density=0.8562, stepsize=0.00004):
    scatters = 0
    # now to run the simulation
    e_i = e_0
    while e_i > epsilon:
        if scattering_probability(e_i, stepsize) > random.random():
            # here, we need to figure out the change inscattering_angle momentum and energy due to the scattering
            # both the proton and alpha particle will cause an event, so we need
            # to tally both
            #transfer_e = energy_transfer(e_i)
            #print(f"Scattering! Angle{transfer_e[2]}")
            scatters += 1
        else:
            # we don't need to multiply by the density here since the LAB data
            # has that baked in for us
            e_i = e_i - stp_interp(e_i)*stepsize
    return scatters

n = scatter_sim(4.7)

print(n)

0


In [35]:
def run_sim_instance(e_0, stepsize=0.0001, nbins=80, epsilon=0.1, density=0.8562):
    n = scatter_sim(e_0, epsilon=epsilon, density=density, stepsize=stepsize)
    return n 

def sim_wrapper(arg):
    args, kwargs = arg
    return run_sim_instance(*args, **kwargs)

def start_sim(e_0, n_particles, stepsize=0.001, nbins=40, epsilon=0.1, density=0.8562):
    arg = (e_0, )
    kwargs = {'stepsize': stepsize, 'nbins': nbins, 'epsilon': epsilon, 'density': density}
    with Pool(floor((2/3)*cpu_count())) as p:
        p_out = p.map(sim_wrapper, [(arg, kwargs) for i in range(n_particles)])

    print(p_out)
    sim_data = sum(p_out) 
    return sim_data 

sim_data = start_sim(4.4, 40000, nbins=100, stepsize=4E-5)
print(sim_data)